In [ ]:
from datetime import date
import pandas as pd
import covsirphy as cs

def analyze_country_data(eng, country_name):
    
    try:
        
        actual_df, status, _ = eng.subset(geo=(country_name,), variables="SIRF", complement=True)
        print(f"\nСтатус данных для {country_name}:")
        print(status)
        
        
        if actual_df.empty:
            print(f"Нет данных для {country_name}")
            return None
            
        
        dyn_act = cs.Dynamics.from_data(
            model=cs.SIRFModel, 
            data=actual_df, 
            name=country_name
        )
        return dyn_act.register()
    except Exception as e:
        print(f"Ошибка при обработке {country_name}: {str(e)}")
        return None

def main():
    
    eng = cs.DataEngineer()
    print("Загрузка данных из COVID-19 Data Hub...")
    
    
    eng.download(databases=["covid19dh"])
    
    
    print("Обработка данных...")
    eng.all().loc[:, 'Date'] = pd.to_datetime(
        eng.all().loc[:, 'Date'], 
        format='%Y-%m-%d', 
        errors='coerce'
    )
    eng.clean(kinds=["convert_date"])
    eng.transform()
    
    
    main_variables = ["Population", "Susceptible", "Confirmed", "Infected", "Fatal", "Recovered"]
    eng.all(variables=main_variables)
    eng.clean()
    eng.transform()
    
    
    countries = ["France", "Germany", "South Korea", "Japan", "Russia"]
    results = {}
    
    
    for country in countries:
        print(f"\nОбработка данных для {country}...")
        country_data = analyze_country_data(eng, country)
        if country_data is not None:
            results[country] = country_data
            print(f"\nПоследние 5 записей для {country}:")
            print(country_data.tail())
    
    
    if results:
        final_dataset = pd.concat(results.values(), keys=results.keys())
        save_path = 'COVID19DH_SIRF_Parameters.csv'
        final_dataset.to_csv(save_path)
        print(f"\nФинальный датасет сохранен как '{save_path}'")
        print(f"Успешно обработано стран: {len(results)}/{len(countries)}")
    else:
        print("\nНе удалось получить данные ни для одной страны")

if __name__ == "__main__":
    main()